# Evo SDK Common

Evo SDK Common is a Python package that establishes a common framework for use by client libraries that interact
with Evo APIs. This notebook demonstrates how to use the package to interact with Evo APIs.

## ITransport

The `ITransport` interface is used to make HTTP requests to Evo APIs. The `AioTransport` class is an implementation
based on the `aiohttp` library, which is an optional dependency. Different HTTP client libraries can be substituted by
implementing a facade that implements the `ITransport` interface.

Transport objects must be re-entrant so that they can be used by multiple coroutines at the same time. `AioTransport`
uses an internal counter to track the number of places where the transport is being used. When the counter reaches zero,
the underlying HTTP client session is closed, and any related resources are released. The next time the transport is
opened, a new session will be created.

In [ ]:
from evo.aio import AioTransport
from evo.common.utils import BackoffIncremental

# Configure the transport.
transport = AioTransport(
    user_agent="your-app-name",
    max_attempts=3,
    backoff_method=BackoffIncremental(2),
    num_pools=4,
    verify_ssl=True,
)

# Open the transport outside a context manager so that the underlying session is left open. This can save
# time if you are going to make multiple batches of requests in the same area of code. Ideally, the transport should
# be closed when it is no longer needed.
await transport.open()

## Logging in to Evo

The `IAuthorizer` interface is used to authenticate with Evo APIs, by automatically attaching the default headers to
API requests. The `AuthorizationCodeAuthorizer` class is an OAuth implementation of `IAuthorizer`, utilizing a reference OAuth
implementation that is built using the `aiohttp` library. `aiohttp` is an optional dependency, so it must be installed
for the `AuthorizationCodeAuthorizer` implementation to work.

In [ ]:
from evo.oauth import AuthorizationCodeAuthorizer, OAuthConnector

# OAuth client app credentials
# See: https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens
CLIENT_ID = "your-client-id"
REDIRECT_URL = "http://localhost:3000/signin-callback"

authorizer = AuthorizationCodeAuthorizer(
    redirect_url=REDIRECT_URL,
    oauth_connector=OAuthConnector(
        client_id=CLIENT_ID,
        transport=transport,
    ),
)

# Login to the Evo platform.
await authorizer.login()

Alternatively, a client of `client credientials` grant type can use the `ClientCredentialsAuthorizer` for authorization into Evo. This allows for service-to-service requests, instead of user login and redirects. 

In [ ]:
from evo.oauth import ClientCredentialsAuthorizer, EvoScopes, OAuthConnector

CLIENT_NAME = "Your Client Name"
CLIENT_ID = "your-client-id"
CLIENT_SECRET = "your-client-secret"

authorizer = ClientCredentialsAuthorizer(
    oauth_connector=OAuthConnector(
        transport=transport,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
    ),
    scopes=EvoScopes.all_evo,
)

# Authorize the client.
await authorizer.authorize()

## Listing organizations

In most user-facing environments it will be necessary to list the organizations that the user has access to. The
`DiscoveryAPIClient` interacts with the Discovery API to retrieve this information. Simply give it a connector
pointing to the appropriate host, and it will do the rest.

In [ ]:
from evo.common import APIConnector
from evo.discovery import DiscoveryAPIClient

# Select an organization
async with APIConnector("https://discover.api.seequent.com", transport, authorizer) as idp_connector:
    discovery_client = DiscoveryAPIClient(idp_connector)
    organizations = await discovery_client.list_organizations()

# Select the first organization for this example
selected_organization = organizations[0]
print("Selected organization:", selected_organization)

## Listing workspaces

Once an organization has been selected, the next step is to list the workspaces that the user has access to.

We will create a connector targeting the hub URL, which we can reuse later for talking to individual services. The
transport and authorizer objects are also reused.

In [ ]:
from evo.workspaces import WorkspaceAPIClient

# You will only have one hub for your organization
hub = selected_organization.hubs[0]
print(hub)

Once you have a hub, you may proceed with listing workspaces.

In [ ]:
# This connector can be used to connect to any service supported by the hub
hub_connector = APIConnector(hub.url, transport, authorizer)

# List the workspaces
async with hub_connector:
    workspace_client = WorkspaceAPIClient(hub_connector, selected_organization.id)
    workspaces = await workspace_client.list_workspaces()

# Select the first workspace for this example (you may have multiple)
selected_workspace = workspaces[0]
print("Selected workspace:", selected_workspace.display_name)

## Interacting with API clients

The `Workspace` object can generate an `Environment`, which contains the organization and workspace IDs, and can be
used to resolve cache locations. Evo SDK Common does not implement any specific API clients, but it provides
a `BaseAPIClient` type that should be used as a base class for API clients.

The `BaseAPIClient` defines a shared constructor for API clients, as well as convenient cache management via the `cache` property and the `clear_cache()` method. 

In [ ]:
from evo.common import BaseAPIClient

## Interact with a service.
async with hub_connector:
    service_client = BaseAPIClient(selected_workspace.get_environment(), hub_connector)
    ...  # Do something with the service client.